In [3]:
pip install lightning

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.1/802.1 kB 3.8 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tiktoken]1/2 [tiktoken]
Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import math
import random
from typing import List, Tuple
import re
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl
import pandas as pd
import tiktoken
from torch.nn.utils.rnn import pad_sequence


if torch.cuda.is_available():
    print("setting cuda as default!")
    torch.set_default_device("cuda")
    torch.set_default_dtype(torch.float32)
else:
    print("cuda not available")

setting cuda as default!


In [23]:
# DEPRECATED
tokenizer = tiktoken.get_encoding('gpt2')
PAD_ID = 50259
BOS_ID = 50257
EOS_ID = 50258
tokenizer = tiktoken.Encoding(
    name = "gpt2",
    pat_str=tokenizer._pat_str,
    mergeable_ranks=tokenizer._mergeable_ranks,
    special_tokens={
        **tokenizer._special_tokens,
        "<bos>": BOS_ID,
        "<eos>": EOS_ID,
        "<pad>": PAD_ID
    }
)

VOCAB_SIZE = tokenizer.n_vocab
VOCAB_SIZE

50260

In [22]:
from collections import Counter

PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"
BOS_TOKEN = "<bos>"
EOS_TOKEN = "<eos>"

special_tokens = [PAD_TOKEN, UNK_TOKEN, BOS_TOKEN, EOS_TOKEN]

def build_vocab(sentences, min_freq=1):
    counter = Counter()
    for s in sentences:
        counter.update(s.strip().lower().split())
    vocab = special_tokens + [word for word, freq in counter.items() if freq >= min_freq]
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for word, idx in word2idx.items()}
    return word2idx, idx2word


In [23]:
def encode_src(sentence, vocab, max_len=20):
    tokens = sentence.split()
    ids = [vocab.get(tok, vocab[UNK_TOKEN]) for tok in tokens]
    ids = ids[:max_len]
    ids += [vocab[PAD_TOKEN]] * (max_len - len(ids))
    return ids

def encode_tgt(sentence, vocab, max_len=20):
    tokens = sentence.split()
    
    # Decoder input: BOS + tokens
    dec_in = [vocab[BOS_TOKEN]] + [vocab.get(tok, vocab[UNK_TOKEN]) for tok in tokens]
    
    # Target: tokens + EOS
    dec_out = [vocab.get(tok, vocab[UNK_TOKEN]) for tok in tokens] + [vocab[EOS_TOKEN]]
    
    # Pad both
    dec_in = dec_in[:max_len]
    dec_out = dec_out[:max_len]
    
    dec_in += [vocab[PAD_TOKEN]] * (max_len - len(dec_in))
    dec_out += [vocab[PAD_TOKEN]] * (max_len - len(dec_out))
    
    return dec_in, dec_out


In [41]:
df = pd.read_csv("dataset.tcsv", sep="\t", header=None, quoting=3)

src_vocab, src_inv = build_vocab(df[1].values.tolist())
tgt_vocab, tgt_inv = build_vocab(df[0].values.tolist())

pairs = list(zip(df[1], df[0]))  # (Portuguese, English)

In [5]:
# DEPRECATED
def preprocess_data(s: str, is_trgt: bool = False) -> str:
    if not isinstance(s, str):
        s = str(s)
    s = s.strip()
    s = re.sub(r'(\w)([.,!?;:])', r'\1 \2', s)
    s = re.sub(r'\s+', ' ', s).strip()
    if is_trgt:
        s = "<bos> " + s
    else:
        s = s + " <eos>"
    return s

def tokenize_text(text: str, max_length: int) -> List[int]:
    encoded_text = tokenizer.encode(text, allowed_special=tokenizer.special_tokens_set)[:max_length]
    if len(encoded_text) < max_length:
        encoded_text.extend([PAD_ID] * (max_length - len(encoded_text)))
    elif len(encoded_text) > max_length:
        encoded_text = encoded_text[:max_length]
    return encoded_text

In [6]:
# DEPRECATED
preprocess_data("hello, I am simply testing...", False)

'hello , I am simply testing ... <eos>'

In [7]:
# DEPRECATED
class WordDataset(Dataset):
  def __init__(self, path, max_length = 64, lang = "pt"):
    df = pd.read_csv(path, sep="\t", header=None, quoting=3)
    df = df[[0, 1]].dropna().reset_index(drop=True)

    if lang == "pt":
      src_col, tgt_col = 1, 0
    else:
      src_col, tgt_col = 0, 1


    df[tgt_col] = df[src_col].apply(preprocess_data)
    df[tgt_col] = df[tgt_col].apply(preprocess_data)

    self.src_seqs, self.tgt_seqs = [], []

    for _, row in df.iterrows():
      src_ids =  tokenize_text(row[src_col], max_length)
      tgt_ids = tokenize_text(row[tgt_col], max_length)

      if src_ids and tgt_ids:
        self.src_seqs.append(torch.tensor(src_ids, dtype=torch.long))
        self.tgt_seqs.append(torch.tensor(tgt_ids, dtype=torch.long))

  def __len__(self):
      return len(self.src_seqs)

  def __getitem__(self, indx):
      return self.src_seqs[indx], self.tgt_seqs[indx]


In [58]:
class TranslationDataset(Dataset):
    def __init__(self, pairs, src_vocab, tgt_vocab, max_len=32):
        self.pairs = pairs
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        src, tgt = self.pairs[idx]
        src_ids = encode_src(src, self.src_vocab, self.max_len)
        dec_in, dec_out = encode_tgt(tgt, self.tgt_vocab, self.max_len)

        return {
            "encoder_input": torch.tensor(src_ids, dtype=torch.long),
            "decoder_input": torch.tensor(dec_in, dtype=torch.long),
            "decoder_target": torch.tensor(dec_out, dtype=torch.long),
        }

In [8]:
# DEPRECATED
def collate_fn(batch: List[Tuple[torch.Tensor, torch.Tensor]]):
    srcs = [b[0] for b in batch]
    tgts = [b[1] for b in batch]
      # Pad to the same length along dim=0 (sequence length)
    srcs = pad_sequence(srcs, batch_first=True, padding_value=PAD_ID)
    tgts = pad_sequence(tgts, batch_first=True, padding_value=PAD_ID)
    return {"src": srcs, "tgt": tgts}

In [9]:
# DEPRECATED
# ======================================================
# 📊 DataLoaders
# ======================================================
def create_word_dataloader(path, lang="pt", max_length=32, batch_size=32, split=0.8):
    dataset = WordDataset(path, max_length=max_length, lang=lang)
    n = len(dataset)
    train_size = int(split * n)
    val_size = n - train_size
    train_ds, val_ds = random_split(dataset, [train_size, val_size], generator=torch.Generator(device='cuda'))
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return train_dl, val_dl

In [65]:
def create_word_dataloader(pairs, src_vocab, tgt_vocab, lang="pt", max_len=32, batch_size=32, split=0.8):
    dataset = TranslationDataset(pairs=pairs, src_vocab=src_vocab, tgt_vocab=tgt_vocab, max_len=max_len)
    n = len(dataset)
    train_size = int(split * n)
    val_size = n - train_size
    train_ds, val_ds = random_split(dataset, [train_size, val_size], generator=torch.Generator(device='cuda'))
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device='cuda'))
    val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False, generator=torch.Generator(device='cuda'))
    return train_dl, val_dl


In [51]:
# ======================================================
# ⚙️ Encoder & Decoder
# ======================================================
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size=256, hidden_size=512, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers=num_layers, batch_first=True)

    def forward(self, src):
        emb = self.embedding(src)
        outputs, hidden = self.gru(emb)
        return outputs, hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size=256, hidden_size=512, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, tgt_inputs, hidden):
        emb = self.embedding(tgt_inputs)
        outputs, hidden_out = self.gru(emb, hidden)
        logits = self.fc(outputs)
        return logits, hidden_out


In [11]:
# DEPRECATED
# ======================================================
# 🚀 Lightning Seq2Seq Model
# ======================================================
class Seq2SeqEncoderDecoder(pl.LightningModule):
    def __init__(self, encoder, decoder, tgt_pad=PAD_ID, lr=1e-3):
        super().__init__()
        self.save_hyperparameters(ignore=["encoder", "decoder"])
        self.encoder = encoder
        self.decoder = decoder
        self.tgt_pad = tgt_pad
        self.lr = lr
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=self.tgt_pad)

    def forward(self, src, tgt_input):
        _, hidden = self.encoder(src)
        logits, _ = self.decoder(tgt_input, hidden)
        return logits

    def training_step(self, batch, batch_idx):
        src, tgt = batch["src"], batch["tgt"]
        src = src.to(self.device)
        tgt = tgt.to(self.device)
        decoder_input = torch.zeros_like(tgt)
        decoder_input[:, 1:] = tgt[:, :-1]
        logits = self(src, decoder_input)
        loss = self.loss_fn(logits.view(-1, logits.size(-1)), tgt.view(-1))
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        src, tgt = batch["src"], batch["tgt"]
        src = src.to(self.device)
        tgt = tgt.to(self.device)
        decoder_input = torch.zeros_like(tgt)
        decoder_input[:, 1:] = tgt[:, :-1]
        logits = self(src, decoder_input)
        loss = self.loss_fn(logits.view(-1, logits.size(-1)), tgt.view(-1))
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


In [29]:
class Seq2SeqEncoderDecoder(pl.LightningModule):
    def __init__(self, encoder, decoder, tgt_pad=0, lr=1e-3):
        super().__init__()
        self.save_hyperparameters(ignore=["encoder", "decoder"])
        self.encoder = encoder
        self.decoder = decoder
        self.tgt_pad = tgt_pad
        self.lr = lr
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=self.tgt_pad)

    def forward(self, src, tgt_input):
        _, hidden = self.encoder(src)
        logits, _ = self.decoder(tgt_input, hidden)
        return logits

    def training_step(self, batch, batch_idx):
        src, tgt_in, tgt_out = batch["encoder_input"], batch["decoder_input"], batch["decoder_target"]
        src = src.to(self.device)
        tgt_in = tgt_in.to(self.device)
        tgt_out = tgt_out.to(self.device)
        logits = self(src, tgt_in)
        loss = self.loss_fn(logits.view(-1, logits.size(-1)), tgt_out.view(-1))
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        src, tgt_in, tgt_out = batch["encoder_input"], batch["decoder_input"], batch["decoder_target"]
        src = src.to(self.device)
        tgt_in = tgt_in.to(self.device)
        tgt_out = tgt_out.to(self.device)
        logits = self(src, tgt_in)
        loss = self.loss_fn(logits.view(-1, logits.size(-1)), tgt_out.view(-1))
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


In [ ]:
# DEPRECATED 
# ======================================================
# 🏋️ Training
# ======================================================
# dataset_path = "/content/dataset.tcsv"  # <-- your dataset path
dataset_path = "dataset.tcsv" 
train_dl, val_dl = create_word_dataloader(dataset_path, lang="pt")


In [66]:
train_dl, val_dl = create_word_dataloader(pairs, src_vocab, tgt_vocab)

In [ ]:
encoder = Encoder(len(src_vocab))
decoder = Decoder(len(tgt_vocab))
model = Seq2SeqEncoderDecoder(encoder, decoder, tgt_pad=0, lr=5e-3)

trainer = pl.Trainer(max_epochs=30, gradient_clip_val=1)
trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=val_dl)


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | encoder | Encoder          | 11.5 M | train
1 | decoder | Decoder          | 20.4 M | train
2 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
32.0 M    Trainable params
0         Non-trainable params
32.0 M    Total params
127.881   Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [16]:
# Save weights only
torch.save(model.state_dict(), "seq2seq_weights.pth")


In [41]:
def translate_sentence(model, sentence: str, max_length: int = 32):
    model.eval()
    with torch.no_grad():
        # Tokenize the input sentence
        src_ids = torch.tensor([tokenize_text(sentence, max_length)], dtype=torch.long)

        # Encode
        _, hidden = model.encoder(src_ids)

        # Start decoding with a "dummy" token (GPT-2 doesn't have BOS, we use PAD_ID or 50256)
        decoder_input = torch.tensor([[BOS_ID]], dtype=torch.long)

        generated_ids = []
        for _ in range(max_length):
            logits, hidden = model.decoder(decoder_input, hidden)
            next_token = torch.argmax(logits[:, -1, :], dim=-1)
            if next_token.item() == PAD_ID:  # stop if PAD_ID token is predicted
                break
            generated_ids.append(next_token.item())
            decoder_input = torch.cat([decoder_input, next_token.unsqueeze(0)], dim=1)

        # Decode token ids back to text
        translated_text = tokenizer.decode(generated_ids)
        return translated_text


In [45]:
test_sentence = "I love"
translation = translate_sentence(model, test_sentence)
print("Portuguese:", test_sentence)
print("English:", translation)

Portuguese: I love
English: gunto ? <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> 


In [43]:
# Recreate the model architecture first
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = Encoder(VOCAB_SIZE)
decoder = Decoder(VOCAB_SIZE)
model = Seq2SeqEncoderDecoder(encoder, decoder, tgt_pad=PAD_ID, lr=5e-3)

# Load weights
model.load_state_dict(torch.load("seq2seq_weights.pth"))
model.eval()  # set to evaluation mode

Seq2SeqEncoderDecoder(
  (encoder): Encoder(
    (embedding): Embedding(50260, 256, padding_idx=50259)
    (gru): GRU(256, 512, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(50260, 256, padding_idx=50259)
    (gru): GRU(256, 512, batch_first=True)
    (fc): Linear(in_features=512, out_features=50260, bias=True)
  )
  (loss_fn): CrossEntropyLoss()
)

In [48]:
df = pd.read_csv("dataset.tcsv", sep="\t", header=None, quoting=3)
df = df[[0, 1]].dropna().reset_index(drop=True)

In [49]:
df.head()

,0,1
0,Go.,Vai.
1,Go.,Vá.
2,Hi.,Oi.
3,Run!,Corre!
4,Run!,Corra!
